### Amazon Book Review

Introduction balallala


0. Data loading
1. Preprocessing
1.1 Preprocessing Minhashing

2. Create Embeddings
2.1 Minhashing
2.2 SBert

3. Clustering for Minhashing
3.1 K-Means
3.2 HDBscan with UMAP

4. Clustering with SBert
4.1 Preprocessing for Cluster
4.2 K-Means
4.3 HDBSCAN

5. Analysis




#### 0. Data loading

In [ ]:
# load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import html
import collections
from collections import Counter

import os
import sys
import argparse
import logging
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq
from sentence_transformers import SentenceTransformer

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [ ]:
# load data
path_book_reviews = "Books_rating.csv" #### SET THE PATH
br = pd.read_csv(path_book_reviews)

#### 1. Preprocessing

##### 1.1 Preprocessing 

In [ ]:
#filter columns
br = br[['Id','review/score', 'review/text']]
#drop NA values
br = br.dropna(subset=['review/text', 'review/score'])
#unescape html characters
br['cleanText'] = br['review/text'].astype(str).apply(html.unescape)

In [ ]:
# --- Combined cleaning function ---
def clean_text_all(text):
    text = str(text)

    # 1. Remove control characters (non-printable ASCII)
    # \x00-\x1F are control chars, \x7F is DEL
    text = re.sub(r'[\x00-\x1F\x7F]', '', text)

    # 2. Remove URLs (http/https/www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # 3. Replace newlines, carriage returns, tabs with a single space
    text = re.sub(r'[\r\n\t]+', ' ', text)

    # 4. Normalize multiple spaces to one and strip leading/trailing spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# --- Apply to your dataframe ---
br['cleanText'] = br['cleanText'].apply(clean_text_all)

In [ ]:
# Remove rows where 'cleanText' is empty after cleaning
br = br.dropna(subset=['cleanText'])

In [ ]:
# Drop all rows that contain non-ASCII characters
# Create a boolean mask for rows that contain any non-ASCII characters
mask_non_ascii = br['cleanText'].str.contains(r'[^\x00-\x7F]', regex=True)

# Print how many will be dropped
print(f"Rows containing non-ASCII characters: {mask_non_ascii.sum()}")

# Drop those rows
br_clean = br[~mask_non_ascii].copy()

In [ ]:
# replace \' with ' and \" with " "
br_clean['cleanText'] = br_clean['cleanText'].str.replace(r"\\'", "'", regex=True)
br_clean['cleanText'] = br_clean['cleanText'].str.replace(r'\\"', '"', regex=True)

In [ ]:
br_clean[['Id', 'review/score', 'cleanText']].to_csv('Books_rating_preprocessed.csv', index=False)

##### 1.2 Preprocessing Minhashing

#### 2. Create Embeddings

##### 2.1 Embeddings Minhasing 

##### 2.2 Embeddings SBert 

In [ ]:
def as_arrow_table(ids, texts, score, embs, sent_scores):
    arr_ids = pa.array(ids)
    arr_texts = pa.array(texts, type=pa.string())
    arr_scores = pa.array(score)
    list_of_lists = [emb.tolist() for emb in embs]
    arr_embs = pa.array(list_of_lists, type=pa.list_(pa.float32()))
    
    arr_sent = pa.array(sent_scores.tolist(), type=pa.float32())
    return pa.Table.from_arrays(
        [arr_ids, arr_scores, arr_texts, arr_embs, arr_sent],
        names=["row_id", "review/score", "cleanText", "embedding", "sentiment_score"]

    )

In [ ]:
def compute_sentiment_scores(texts, tokenizer, model, device, batch_size=64, max_length=256):

    scores = []
    model.eval()

    with torch.no_grad():
        for start in range(0, len(texts), batch_size):
            batch_texts = texts[start:start + batch_size]
            enc = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=max_length
            )
            enc = {k: v.to(device) for k, v in enc.items()}

            outputs = model(**enc)
            probs = F.softmax(outputs.logits, dim=1)  # shape: [B, 5] (for 1–5 stars)

            stars = torch.arange(1, probs.size(1) + 1, device=device, dtype=torch.float32)
            batch_scores = (probs * stars).sum(dim=1) 
            scores.extend(batch_scores.cpu().numpy().tolist())

    return np.array(scores, dtype=np.float32)

In [ ]:
# Create SBERT and Sentiment Analysis models
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

sent_tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment" )
sent_model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

NameError: name 'SentenceTransformer' is not defined

In [ ]:
reader = pd.read_csv(path_book_reviews, sep=",", encoding="utf-8", dtype=str, keep_default_na=False,on_bad_lines="skip")

In [ ]:
# get texts and scores
texts = reader["cleanText"].astype(str).tolist()
score = reader["Id"].astype(str).tolist()

In [ ]:
embs = model.encode(
                texts,
                convert_to_numpy=True,
                show_progress_bar=True
            ).astype(np.float32)


In [ ]:
sentiment_scores = compute_sentiment_scores(
                texts,
                sent_tokenizer,
                sent_model,
                batch_size=64,       
                max_length=256
            )

In [ ]:
# add embeddings and sentiment scores and data to arrow table
table = as_arrow_table(
    ids=reader["Id"].astype(str).tolist(),
    texts=reader["cleanText"].astype(str).tolist(),
    score=reader["review/score"].astype(str).tolist(),
    embs=embs,
    sent_scores=sentiment_scores
)
pq.write_table(table, "book_reviews_with_embeddings.parquet")


#### Clustering for Minhashing 

#### Clustering with SBert

### Analysis